From [wrangle report](https://github.com/wrayz/hw-wrangle-and-analyze-data/blob/master/wrangle_report.ipynb) I wrangling three pieces of information related to the [@WeRateDogs](https://twitter.com/dog_rates) tweet into a single file. `twitter_archive_master.csv`, the following is my analysis

Recent tweets

![Cute](./images/dog.png)

In [2]:
# Top five data
import pandas as pd
pd.read_csv('twitter_archive_master.csv').head()

,tweet_id,text,expanded_urls,name,favorite_count,lang,retweet_count,source,created_time,rating_ratio,stage,breed
0,892177421306343426,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,32434,en,6079,iPhone,2017-08-01 00:17:27,1.3,None,Chihuahua
1,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,24430,en,4023,iPhone,2017-07-31 00:18:03,1.2,None,Chihuahua
2,891689557279858688,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,41088,en,8371,iPhone,2017-07-30 15:58:51,1.3,None,Labrador_retriever
3,891327558926688256,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,39282,en,9082,iPhone,2017-07-29 16:00:24,1.2,None,basset
4,891087950875897856,Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,None,19757,en,3008,iPhone,2017-07-29 00:08:17,1.3,None,Chesapeake_Bay_retriever


In this data, I am curious about two simple questions:

**_Q1: Is @WeRateDogs' tweet evaluation the key to popular?_**

**_Q2: If not, what is the reason for the tweet to let the public report or praise?_**

First let's look at the data distribution of the @WeRateDogs rating ratio.

![Box](./images/box_ratingratio.png)

From the box plot display, the @WeRateDogs score has no specific pattern, it is difficult to directly confirm that the score is the reason why the tweet received everyone's favorite; in addition, because there is no specific pattern for the score, we can't directly score the score to compare different dogs.

Obviously we have to go into the second question. Here I have a definition of popular. The more tweets I have, the more popular the favorite count or retweet count is.

The following can be seen from the heat map. The favorite count and retweet count are positive correlations. Obviously everyone will like and share the favorite posts. So I will observe this on the basis of the favorite count and ask: 

**_Which feature will make the tweet get more favorite count?_**

Also unfortunately, the heat map shows that @WeRateDog tweet's rating ratio has no strong relationship with favorite count and retweet count. And this result reinforces the speculation that began with the distribution of data from the @WeRateDog rating ratio.


![HeatMap](./images/heatmap.png)

We select breed, stages, created time, and favorite count for visual comparison.

1. Breed

    First, we get the number of different data in different breed, and then look at the number of favorites in different categories, and then calculate the correlation coefficient of the two sets of data, the result is up to $0.99$, which means that the number of favorite and tweets in breed is very high. Big relationship.

    At the same time, we use the bar graph to show the number of favorites in different categories. It can be seen that the top 3 are Golden retriever, Labrador retriever, Pembroke, perhaps the current popular category, and there will be so many tweets and Favorites.

![Bar](./images/barh_breed.png)

2. Stages of Dog

    Does a different and even more diverse stage affect the favorite count?

    The histogram shows that most of the favorite counts fall on 'None', which is an unrecognizable stage. The relationship between stage and favorite cannot be seen from the data.

![StageBar](./images/bar_stages.png)

3. Created Time

    We cut the time and count the favorite counts obtained every hour according to 24 hours. The line chart shows that during the specific time period, the favorite count is particularly high, obviously most users have a common time to go online and give favorite. Bingo！

![Trend](./images/trend_time.png)

## Conclusion

From the data analysis, we know a few things
1. @WeRateDogs scores do not have a certain standard, and it does not directly affect the user's preferences.
2. Different dog breeds are not directly related to favorite counts, but because some breeds of dogs have more tweets, so there are more accumulated favorite counts.
3. Whether the tweet can get more favorite counts, the data display is related to the general user's specific online time.